In [ ]:
#source EPOTFenv/bin/activate (launching and exiting the virtual environment containing the required modules, stored in the working directory)
#EPOTFenv/bin/python your_script.py - for running a script in the virtual environment
#source deactivate

#import all the libraries needed
from import_dep import *

# import my functions stored in the file 'functions' which can then be called with functions.function_name()
import functions



# Directories for each Device Measured
directory_1 =  Path('/Users/horatiocox/Desktop/RUG Postdoc/Experiments/Electrical/VDP_JT029_SrRuO3/')
directory_2 =  Path('/Users/horatiocox/Desktop/RUG Postdoc/Experiments/Electrical/VDP_HC003_BaSnO3/')
directory_3 =  Path('/Users/horatiocox/Desktop/RUG Postdoc/Experiments/Electrical/VDP_YFW042_HZO/')


############## User Defined Variables ##############


###Plotting and Output Variables 
plot_data = False # set to True to plot the data and export to path_out
powerpoint_data = False # set to True to export the data to a powerpoint presentation and export to path_out
fig_format = 'png' # format of the figure to be saved
plot_transparency = True # set to True to make the data plots transparent

############## End User Defined Variables ##############


##########TO DO LIST
# change so that if film_thickness == 1 all the functions output the sheet resistance
# and the plots show sheet resistance instead of resistivity
# including the output file names
# - SET to zero for sheet resistance calculations instead of resistivity??????

# check and sort the material title and plotstrings in a way that makes sense - perhaps appending them together?
# update so that i only have one set of markers, defined here and used throughout. Perhaps with colors it should be generated with the
# data extraction matching the field and current values to the markers and colors


############## End User Defined Variables ##############


## Import Measurement Data 

In [ ]:
# Initialise empty lists to store input variables for each data set
film_thickness = [] # film thicknesses in meters (m) - Set to 1 to calculate sheet resistance instead of resistivity for 2DEG etc 
material_title = [] # materials to be added to plot titles
plot_string = [] # strings to be used at the start of the figure file name, including the sample code
d_in = [] # directories for the data to be imported from
file_names = [] # file names for the data to be imported from

### First data set
d_in.append(directory_3)
file_names.append(Path('YF042_HfO2_STO_071124_Hall_6'))
film_thickness.append(1e-9)
material_title.append('$HfO_2$')
plot_string.append('YF042_HfO2')

path_out = d_in[0].joinpath('Output/') # Output directory to save plots and powerpoint to - always defined by the first data set

### Second data set
d_in.append(directory_3)
file_names.append(Path('YF042_HfO2_STO_071124_Hall_5'))
film_thickness.append(1e-9)
material_title.append('$HfO_2$')
plot_string.append('YF042_HfO2')

# Initialise empty lists to store the imported data
data_import_np = []
data_import_df = []

# Loop through the data sets and import the data
for j in range(len(file_names)):
    np_data, df_data = functions.import_ppms_data(d_in[j].joinpath('Data/').joinpath(file_names[j]))
    data_import_np.append(np_data)
    data_import_df.append(df_data)
    print(f'np_{j} shape:', data_import_np[j].shape)
    print(data_import_df[j].head(6))

# Note that field has been converted to Tesla

FileNotFoundError: [Errno 2] No such file or directory: '/Users/horatiocox/Desktop/RUG Postdoc/Experiments/Electrical/VDP_YFW042_HZO/Data/YF042_HfO2_STO_071124_Hall_6'

## Extract and Check Temperature, Field, and Current values

In [ ]:
# Extract the current, temperature and field values from the data as lists along with the number of values used for each
#ctf = [current_unique, temp_unique, field_unique, current_no, temp_no, field_no]
# if re-ordering is true then we reassign data_np to order the fields from -Bmax to Bmax
# Reduced_data = [1,-5] means that the first and last 5 temperature points are removed from the data

# Initialise empty lists to store the extracted data
ctf = []
tf_av = []
data_np = []
data_df = []

for j in range(len(data_import_np)):
    [ctf_j, tf_av_j, data_np_j, data_df_j] = functions.extract_ctf(data_import_np[j], Reduced_temp = False, Reduced_current=False)
    ctf.append(ctf_j)
    tf_av.append(tf_av_j)
    data_np.append(data_np_j)
    data_df.append(data_df_j)

    # Check you have the expected and correct Temperature, Field and Current values!
    print(f'np({j}) shape:', data_np[j].shape,'---------------------------------------------------------------')
    print(data_df[j].head())


## Calculate Van der Pauw Resistivity 

In [ ]:
# Initialise empty lists to store the extracted data
res_data = []
res_data_df = []
R_squared_res = []

for j in range(len(data_import_np)):
    [res_data_j, res_data_df_j, R_squared_res_j] = functions.vdp_resistivity(data_np[j], film_thickness[j], ctf[j], tf_av[j])
    res_data.append(res_data_j)
    res_data_df.append(res_data_df_j)
    R_squared_res.append(R_squared_res_j)

    print(f'res_data({j})')
    print(res_data_df[j].head())


## Plot IV data to Check Linearity of Resitivity Measurements

In [ ]:
fig0 = plt.figure(figsize=(14, 3*len(data_import_np)))
gs = fig0.add_gridspec(len(data_import_np), 2)
labels = ['index_3', 'index_4', 'index_5', 'index_6']
color_s = ['r', 'g', 'b', 'm'] # simple four colors for the 4 configurations of vdp

for j in range(len(data_import_np)):
    
    ax = fig0.add_subplot(gs[j, 0])
    ax1 = fig0.add_subplot(gs[j, 1])

    ## Plotting Source(I) vs Sense(V) data
    for i in range(4):
        R_test = linregress(data_np[j][0:ctf[j][3], 2, 2+i], data_np[j][0:ctf[j][3], 4, 2+i])
        ax.plot(data_np[j][0:ctf[j][3], 2, 2+i], data_np[j][0:ctf[j][3], 4, 2+i], 'o', label=f'{labels[i]}', color=color_s[i])
        ax.plot(data_np[j][0:ctf[j][3], 2, 2+i], R_test.intercept + R_test.slope * data_np[j][0:ctf[j][3], 2, 2+i], color=color_s[i], label=f'{labels[i]}')
    ax.legend(loc='best', fontsize='small', markerscale=0.8, framealpha=0.4)

    # Set labels and title
    ax.set_xlabel('Source Current (A)')
    ax.set_ylabel('Sense Voltage (V)')
    ax.set_title(f'IV Sense: H = {np.round(data_np[j][0, 1, 2],decimals=0)} kOe')


    ## Plotting the Source IV data
    colors = plt.get_cmap('winter')(np.linspace(0, 1, ctf[j][4]))
    markers = ['+', 'x', '^', 'd']
    for t in range(ctf[j][4]):
        counter = t*ctf[j][3]*ctf[j][5]
        for i in range(4):  
            R_test_2 = linregress(data_np[j][counter:ctf[j][3]+counter, 2, 2], data_np[j][counter:ctf[j][3]+counter, 3, 2+i])
            ax1.scatter(data_np[j][counter:ctf[j][3]+counter, 2, 2+i], data_np[j][counter:ctf[j][3]+counter, 3, 2+i], color=colors[t], marker=markers[i])
            ax1.plot(data_np[j][counter:ctf[j][3]+counter, 2, 2+i], R_test_2.intercept + R_test_2.slope * data_np[j][counter:ctf[j][3]+counter, 2, 2+i], color=colors[t])
        
        # Add a dummy plot for the legend entry
        ax1.plot([], [], color=colors[t], label=f'T = {ctf[j][1][t]} K')
        
        ax1.legend(loc='best', fontsize='small', markerscale=0.8, framealpha=0.4)

    # Set labels and title
    ax1.set_xlabel('Source Current (A)')
    ax1.set_ylabel('Source Voltage (V)')
    ax1.set_title(f'IV Source: H = {np.round(data_np[j][0, 1, 2],decimals=0)} kOe')


plt.show()

if plot_data == True:
    fig0.savefig(Path(path_out).joinpath(f'{plot_string}_IV_resistivity.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)


## R-squared Analysis to Check Linearity of I-V data

In [ ]:
fig1 = plt.figure(figsize=(15, 3*len(data_import_np) // 2))
gs = fig1.add_gridspec(1+len(data_import_np) // 2, 2)

for j in range(len(data_import_np)):
    print(f'average R-squared for data_{j} =', sum(R_squared_res[j]) / len(R_squared_res[j]))
    ax = fig1.add_subplot(gs[j // 2, j % 2])
    
    # Plot R-square values in a boxplot
    # Note that the y axis is showing the deviation in 1e-6 from the perfect value of 1 so generally the values are very close to 1
    ax.boxplot(R_squared_res[j])
    ax.set_title(f'Box Plot of R-squared Values for data_{j}')
    ax.set_ylabel('R-squared Value')
    ax.set_ylim(1 - 1e-3, 1)  # If needed, you can set x-axis limits too

plt.show()


## Resistivity Configuration Compararison 
Comparing the resistivity in configuration A and B which are inverted versions of one another (e.g. source meter and sense meter swapped positions)
This is a good check for any systematic errors

In [ ]:
# Create a figure and axis
fig2 = plt.figure(figsize=(10, 6))
ax = fig2.add_subplot(111)  # Create a single subplot

#loop over the data sets
for j in range(len(data_import_np)):
    
    # Plot R_sheet_A vs Temp
    ax.scatter(res_data[j][:,0], res_data[j][:,2]* 1e8 , marker='s', color='r', label = r'($\rho_A$)')

    # Plot R_sheet_B vs Temp
    ax.scatter(res_data[j][:,0], res_data[j][:,3]* 1e8 , marker='^', color='g', label = r'($\rho_B$)')

# Set labels and title
ax.set_xlabel('Temperature (K)')
ax.set_ylabel(r'Resistivity ($\mu \Omega \cdot$cm)')
ax.set_title('Resistivity vs Temperature')

# Add a grid
ax.grid(True)

# Add a legend
ax.legend(loc = 'best', framealpha=0.4)

# Show the plot
plt.show()

if plot_data == True:
    fig2.savefig(Path(path_out).joinpath(f'{plot_string}_rho_xx_A_vs_B.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)


## Resistivity vs Temperature

In [ ]:
# Create a figure and axis
fig3 = plt.figure(figsize=(10, 6))
ax = fig3.add_subplot(111)  # Create a single subplot



#loop over the data sets
for j in range(len(data_import_np)):
    #c_val = plt.cm.jet(np.linspace(0,1,num_field_points))
    # Generating one set of colors for increasing positive field and one set for decreasing negative field
    # higher field is brighter color in both cases
    cmap1 = plt.get_cmap('winter')(np.linspace(0, 1, int(ctf[j][5] / 2)))
    cmap2 = plt.get_cmap('Greys')(np.linspace(0.2, 1, int(ctf[j][5] / 2)))
    c_val = np.vstack((cmap1, cmap2))

    # Create separate marker arrays for positive and negative fields stiched together
    markers = ['x'] * int(ctf[j][5] / 2) + ['+'] * int(ctf[j][5] / 2)

    for k in range(ctf[j][5]):
        # Plot rho_film vs Temp
        ax.scatter(res_data[j][k::ctf[j][5],0], res_data[j][k::ctf[j][5],4] * 1e8, marker=markers[k], color=c_val[k], label = f'H={np.round(res_data[j][k,1]*10,decimals=0)} kOe')

# Set labels and title
ax.set_xlabel('Temperature (K)')
ax.set_ylabel(r'$\rho\,(\mu \Omega \cdot cm)$')
ax.set_title(f'Resistivity of {material_title[j]} vs Temperature')
# ax.set_xlim(0,100)
#ax.set_ylim(0,15)

# Add a grid
ax.grid(True)

# Add a legend
ax.legend(loc = 'best', framealpha=0.4)

# ax.set_xlim(200, 250)
#ax.set_ylim(0, 0.09)

# Show the plot
plt.show()

if plot_data == True:
    fig3.savefig(Path(path_out).joinpath(f'{plot_string}_rho_xx_vs_temperature.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)  


- Anderson localisation causes the metal insulator transition at low T
- If you have defects in film, they can have an external effect of localising charge carriers and reducing the conductivity
    - Also causes loss of metalicity beneath a certain thicknes of LSMO etc
    -  Defects localise the previously delocalised charge carriers, either due to thickness reudction or low T
    - Low thicknes its because number of defects is constant - > higher defect density
    

## Magnetoresistance vs H
- Plotting the magnetoresistance vs the Applied field strength (H) for several temperatures

In [ ]:
# Calculate the magnetoresistance at each temperature and field strength
# mag_res(temperature, field, index) where: index 0 = VDP config A, index 1 = VDP config B, index 2 = average of A and B
mag_res = []
for i in range(len(data_import_np)):
    mag_res_i = functions.magnetoresistance(data_np[i], film_thickness[i], ctf[i], tf_av[i])
    mag_res.append(mag_res_i)

fig4 = plt.figure(figsize=(10, 6 * len(data_import_np)))
gs = fig4.add_gridspec(len(data_import_np), 1)
ax = fig4.add_subplot(gs[j, 0])
for j in range(len(data_import_np)):
    
    
    # Create a color map for the temperature points
    c_val = plt.cm.jet(np.linspace(0, 1, ctf[j][4]))
    # Create a marker array for the temperature points
    markers = ['+', '.', '*', ',', 'o', 'v', '^', '<', '>', '1', '2', '3', '4', 's', 'p', 'h', 'H', 'x', 'D', 'd', '|', '_']

    for t_count, t in enumerate(ctf[j][1], start=0):  
        if t > 1:
            # Add a plot of magnetoresistance vs field strength for each loop of temperature (excluding last field point which is repeat of B=0)
            ax.scatter(ctf[j][2][:] * 10, mag_res[j][t_count, :, 2], marker=markers[j], color=c_val[t_count], label=f'T={t} K')
            # Plot repeat point of B=0 with an x marker to show the difference between two B=0 points
            # ax.scatter(ctf[j][2][int(ctf[j][5] / 2)] * 10, mag_res[j][t_count, int(ctf[j][5] / 2), 2], marker='x', color=c_val[t_count])
    
ax.set_xlabel('H (kOe)')
ax.set_ylabel('Magnetoresistance (%)')
ax.set_title(f'Magnetoresistance vs Field for Dataset {j+1}')
ax.legend(loc='best', framealpha=0.4)
ax.set_ylim(-100, 100)

plt.show()

if plot_data == True:
    fig4.savefig(Path(path_out).joinpath(f'{plot_string[j]}_magnetoresistance_vs_field.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)


In [ ]:
print(int(ctf[0][5]/2))
ctf[0][2][int(ctf[0][5]/2)]

## Magnetoresistance Vs T
- Now looking at how the Magnetoresitance at a set field strength varies over temperature
- Displaying this for VDP configuration A, VDP configuration B and the average of the two

In [ ]:
# Calculate the magnetoresistance at each temperature and field strength
# mag_res(temperature, field, index) where: index 0 = VDP config A, index 1 = VDP config B, index 2 = average of A and B
mag_res = functions.magnetoresistance(data_np, film_thickness, ctf, tf_av)

fig5 = plt.figure(figsize=(15, 6))
ax = fig5.add_subplot(121)  # Create two subplot
ax2 = fig5.add_subplot(122)  


# Generating one set of colors for increasing positive field and one set for decreasing negative field
# higher field is brighter color in both cases
cmap1 = plt.get_cmap('jet')(np.linspace(0, 1, int(ctf[5] / 2)))
cmap2 = plt.get_cmap('jet_r')(np.linspace(0, 1, int(ctf[5] / 2)))
c_val = np.vstack((cmap1, cmap2))

# Create separate marker arrays for positive and negative fields stiched together
markers = ['*'] * int(ctf[5] / 2) + ['+'] * int(ctf[5] / 2)

# Find the index of the maximum positive field strength
f_index = int(ctf[5])-1#int(ctf[5]/2 - 1)

# Sub_plot 1: Comparing configurations A and B
ax.plot(ctf[1], mag_res[:,f_index,0], marker='o', color=c_val[0], label = f'VDP_A - H={np.round(ctf[2][f_index]/1000,0)} kOe')
ax.plot(ctf[1], mag_res[:,f_index,1], marker='x', color=c_val[int(len(c_val)/2)], label = f'VDP_B - H={np.round(ctf[2][f_index]/1000,0)} kOe')

ax.set_xlabel('Temperature (K)')
ax.set_ylabel(r'Magnetoresistance (%)')
ax.set_title(r'$100 \cdot \frac{\rho_{B} - \rho_{0}}{\rho_{0}}$ vs Temperature')
ax.legend(loc = 'best', framealpha=0.4)

# Sub_plot 2: Comparing the average magnetoresistance vs T for various field strengths 

for H_count, H in enumerate(ctf[2], start=0):
    ax2.plot(ctf[1], mag_res[:,H_count,2], marker=markers[H_count], color=c_val[H_count], label = f'Average - H={np.round(ctf[2][H_count]*10,0)} kOe')     

ax2.set_xlabel('Temperature (K)')
ax2.set_ylabel(r'Magnetoresistance (%)')
ax2.set_title(r'$100 \cdot \frac{\rho_{B} - \rho_{0}}{\rho_{0}}$ vs Temperature')
# Add a legend outside the plot
ax2.legend(loc='lower right', fontsize='small', markerscale=0.8, framealpha=0.4)

# Adjust the plot to make room for the legend
#plt.tight_layout(rect=[0, 0, 0.75, 1])

ax2.set_ylim(-100,100)

plt.show()

if plot_data == True:
    fig5.savefig(Path(path_out).joinpath(f'{plot_string}_magnetoresistance_vs_temperature.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)

## Arhenius Plot

In [ ]:
# Create a figure and axis
fig6 = plt.figure(figsize=(10, 6))
ax = fig6.add_subplot(111)  # Create a single subplot

# Plot rho_film vs 1000/Temp
ax.scatter(1000 / res_data[int(ctf[5]/2)::ctf[5],0], np.log(res_data[int(ctf[5]/2)::ctf[5],4]), marker='o', linestyle='-', color='b', label='Ln Resistivity')

# Set labels and title
ax.set_xlabel(r'$1000/T$ (K$^{-1}$)')
ax.set_ylabel(r'$\ln(\rho)$ ($\Omega \cdot m$)')
ax.set_title('Arrhenius Plot at H = 0 kOe')

# Add a grid
ax.grid(True)

# Add a legend
ax.legend(loc = 'best', framealpha=0.4)

# Create a secondary x-axis at the top
ax2 = ax.twiny()

# Set the limits of the secondary x-axis to match the primary x-axis
ax2.set_xlim(ax.get_xlim())

# Set the ticks and labels for the secondary x-axis
new_ticks = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], num=5)
ax2.set_xticks(new_ticks)
ax2.set_xticklabels([f'{1000/tick:.1f}' for tick in new_ticks])
ax2.set_xlabel('Temperature (K)')

# Show the plot
plt.show()

if plot_data == True:
    fig6.savefig(Path(path_out).joinpath(f'{plot_string}_Arrhenius Plot.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)

## Calculate the Hall Resistivity ($\rho_{xy}$)
- This is: $\frac{V_{measured-perpendicular}}{I_{source}}*t$


In [ ]:
[hall_data, hall_data_df, hall_coefficient, hall_coefficient_df] = functions.vdp_hall(data_np, film_thickness, ctf, tf_av)
print(hall_data_df.head(20))

## Plot IV Data to check linearity of Hall measurements

In [ ]:
fig0 = plt.figure(figsize=(14, 5))
gs = fig0.add_gridspec(1, 2)
ax = fig0.add_subplot(gs[0, 0])
ax1 = fig0.add_subplot(gs[0, 1])

labels = ['index_3', 'index_4', 'index_5', 'index_6']

## If you get a problem with regression, uncomment the below code to plot V vs I and check the linearity of the fit manually
colors = ['r', 'g', 'b', 'm']
for i in range(2):
    R_test = linregress(data_np[0:ctf[3], 2, i], data_np[0:ctf[3], 4, i]) 
    ax.plot(data_np[0:ctf[3], 2, i], data_np[0:ctf[3], 4, i], 'o', label=f'{labels[i]}', color=colors[i])
    ax.plot(data_np[0:ctf[3], 2, i], R_test.intercept + R_test.slope * data_np[0:ctf[3], 2, i], color=colors[i], label=f'{labels[i]}')
ax.legend(loc='best', fontsize='small', markerscale=0.8, framealpha=0.4)

# Set labels and title
ax.set_xlabel('Source Current (A)')
ax.set_ylabel('Sense Voltage (V)')
ax.set_title(f'IV Sense: H = {np.round(data_np[0, 1, 2],decimals=0)} kOe')


## Plotting the Source IV data
colors = plt.get_cmap('winter')(np.linspace(0, 1, ctf[4]))
markers = ['+', 'x', 'o', '*']
for t in range(ctf[4]):
    counter = t*ctf[3]*ctf[5]
    for i in range(2):  
        R_test_2 = linregress(data_np[counter:ctf[3]+counter, 2, i], data_np[counter:ctf[3]+counter, 3, i])
        ax1.scatter(data_np[counter:ctf[3]+counter, 2, i], data_np[counter:ctf[3]+counter, 3, i], color=colors[t], marker=markers[i])
        ax1.plot(data_np[counter:ctf[3]+counter, 2, i], R_test_2.intercept + R_test_2.slope * data_np[counter:ctf[3]+counter, 2, i], color=colors[t])
    
    # Add a dummy plot for the legend entry
    ax1.plot([], [], color=colors[t], label=f'T = {ctf[1][t]} K')
    
    ax1.legend(loc='best', fontsize='small', markerscale=0.8, framealpha=0.4)

# Set labels and title
ax1.set_xlabel('Source Current (A)')
ax1.set_ylabel('Source Voltage (V)')
ax1.set_title(f'IV Source: H = {np.round(data_np[0, 1, 2],decimals=0)} kOe')


plt.show()

if plot_data == True:
    fig0.savefig(Path(path_out).joinpath(f'{plot_string}_IV_resistivity.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)


## R-squared Analysis to Check Linearity of Resistivity I-V data

In [ ]:
print('average R-squared_A =',np.mean(np.abs(hall_data[:,3])))
print('average R-squared_B =',np.mean(np.abs(hall_data[:,5])))

# Plot R-square values in a boxplot
# Note that the y axis is showing the deviation in 1e-6 from the perfect value of 1 so generally the values are very close to 1
fig7 = plt.figure(figsize=(12, 4))
ax = fig7.add_subplot(121)
ax.boxplot(np.abs(hall_data[:,3]))
ax.set_title('Box Plot of R-squared Values - Configuration A')
ax.set_ylabel('R-squared Value')
ax.set_ylim(1-5e-3, 1)  # If needed, you can set x-axis limits too

ax2 = fig7.add_subplot(122)
ax2.boxplot(np.abs(hall_data[:,5]))
ax2.set_title('Box Plot of R-squared Values - Configuration B')
ax2.set_ylabel('R-squared Value')
ax2.set_ylim(1-5e-3, 1)  # If needed, you can set x-axis limits too
plt.show()


### If you get a problem with regression, uncomment the below code to plot V vs I and check the linearity of the fit manually
# R_test = linregress(data_np[0:ctf[3]-1, 2, 2], data_np[0:ctf[3]-1, 4, 2])   
# plt.plot(data_np[0:ctf[3], 2, 2], data_np[0:ctf[3], 4, 2], 'o', label='original data')
# plt.plot(data_np[0:ctf[3], 2, 2], R_test.intercept + R_test.slope * data_np[0:ctf[3], 2, 2], 'r', label='fitted line')
# plt.legend()
# plt.show()

if plot_data == True:
    fig7.savefig(Path(path_out).joinpath(f'{plot_string}_R_squared_hall_res.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)

## Hall resisitivity vs Temperature

In [ ]:
# Create a figure and axis
fig8 = plt.figure(figsize=(10, 6))
ax = fig8.add_subplot(111)  # Create a single subplot

#c_val = plt.cm.jet(np.linspace(0,1,num_field_points))
# Generating one set of colors for increasing positive field and one set for decreasing negative field
# higher field is brighter color in both cases
cmap1 = plt.get_cmap('winter')(np.linspace(0, 1, int(ctf[5] / 2)))
cmap2 = plt.get_cmap('winter_r')(np.linspace(0, 0.8, int(ctf[5] / 2)))
c_val = np.vstack((cmap1, cmap2))

# Create separate marker arrays for positive and negative fields stiched together
markers = ['x'] * int(ctf[5] / 2) + ['+'] * int(ctf[5] / 2)


for k in range(ctf[5]):
    # Plot rho_film vs Temp
    ax.plot(hall_data[k::ctf[5],0], hall_data[k::ctf[5],6]*1e8 , marker=markers[k], color=c_val[k], label = f'H={np.round(hall_data[k,1]*10,decimals=0)} kOe')

# Set labels and title
ax.set_xlabel('Temperature (K)')
ax.set_ylabel(r'$\rho_{xy}\,(\mu \Omega \cdot cm)$')
ax.set_title(r'$\rho_{xy}$' f' of {material_title} vs Temperature')
# ax.set_xlim(0,100)
# ax.set_ylim(1425,1525)

# Add a grid
ax.grid(True)

# Add a legend
ax.legend(loc = 'lower right', framealpha=0.4)

# Show the plot
plt.show()

if plot_data == True:
    fig8.savefig(Path(path_out).joinpath(f'{plot_string}_rho_xy_vs_T.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)

## Hall Resistivity vs Field

In [ ]:
#ctf = [current_unique, temp_unique, field_unique, current_no, temp_no, field_no]

# Create a figure and axis
fig9 = plt.figure(figsize=(10, 6))
ax = fig9.add_subplot(111)  # Create a single subplot

#c_val = plt.cm.jet(np.linspace(0,1,num_field_points))
# Generating one set of colors for increasing positive field and one set for decreasing negative field
# higher field is brighter color in both cases
c_val = plt.get_cmap('winter')(np.linspace(0, 1, ctf[4]))


# Create separate marker arrays for positive and negative fields stiched together
markers = ['x'] * ctf[4]


for t_count, t in enumerate(ctf[1], start=0):
    if t > 1:
        index = t_count * ctf[5]
        # Plot rho_film vs Temp
        ax.plot(hall_data[index:index+ctf[5],1]*10, hall_data[index:index+ctf[5],6]*1e8, marker=markers[t_count], color=c_val[t_count], label = f'T={np.round(t,decimals=0)} K')
# Set labels and title
ax.set_xlabel('H (kOe)')
ax.set_ylabel(r'$\rho_{xy}\,(\mu \Omega \cdot cm)$')
ax.set_title(r'$\rho_{xy}$' f' of {material_title} vs Applied Field')
# ax.set_xlim(0,100)
# ax.set_ylim(1425,1525)

# Add a grid
ax.grid(True)

# Add a legend
ax.legend(loc = 'best', framealpha=0.4, markerscale=0.8, fontsize='xx-small')

# Show the plot
plt.show()

if plot_data == True:
    fig9.savefig(Path(path_out).joinpath(f'{plot_string}_rho_xy_vs_field.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)

## Hall Resistivity vs Field - Offset removed

In [ ]:
#ctf = [current_unique, temp_unique, field_unique, current_no, temp_no, field_no]

# Create a figure and axis
fig10 = plt.figure(figsize=(10, 6))
ax = fig10.add_subplot(111)  # Create a single subplot

#c_val = plt.cm.jet(np.linspace(0,1,num_field_points))
# Generating one set of colors for increasing positive field and one set for decreasing negative field
# higher field is brighter color in both cases
c_val = plt.get_cmap('winter')(np.linspace(0, 1, ctf[4]))


# Create separate marker arrays for positive and negative fields stiched together
markers = ['x'] * ctf[4]


for t_count, t in enumerate(ctf[1], start=0):
    if t > 1:
        index = t_count * ctf[5]
        # Plot rho_film vs Temp
        ax.plot(hall_data[index:index+ctf[5],1]*10, (hall_data[index:index+ctf[5],6]-hall_data[index+int(ctf[5]/2),6])*1e8, marker=markers[t_count], color=c_val[t_count], label = f'T={np.round(t,decimals=0)} K')
    
# Set labels and title
ax.set_xlabel('H (kOe)')
ax.set_ylabel(r'$\rho_{xy}\,(\mu \Omega \cdot cm)$')
ax.set_title(r'$\rho_{xy}$' f' of {material_title} vs Applied Field - Offset Removed')
# ax.set_xlim(0,100)
# ax.set_ylim(1425,1525)

# Add a grid
ax.grid(True)

# Add a legend
ax.legend(loc = 'best', framealpha=0.4, markerscale=0.8, fontsize='xx-small')

# Show the plot
plt.show()

if plot_data == True:
    fig10.savefig(Path(path_out).joinpath(f'{plot_string}_rho_xy_vs_field_0offset.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)

## Hall Coeff Vs T?

In [ ]:
print(hall_coefficient_df.head(10))

In [ ]:


# Create a figure and axis
fig11 = plt.figure(figsize=(10, 6))
ax = fig11.add_subplot(111)  # Create a single subplot

# Generate color map
c_val = plt.get_cmap('winter')(np.linspace(0, 1, 3))

# Create separate marker arrays for positive and negative fields stitched together
markers = ['x','o','*']


ax.scatter(hall_coefficient[:, 0], 1e-6*np.divide(-1,np.multiply(hall_coefficient[:, 1],scipy.constants.e)), marker=markers[0], color=c_val[0], label=f'Config_A')
ax.scatter(hall_coefficient[:, 0], 1e-6*np.divide(-1,np.multiply(hall_coefficient[:, 3],scipy.constants.e)), marker=markers[1], color=c_val[1], label=f'Config_B')
ax.scatter(hall_coefficient[:, 0], 1e-6*np.divide(-1,np.multiply(hall_coefficient[:, 5],scipy.constants.e)), marker=markers[2], color=c_val[2], label=f'Average')

# Set labels and title for the left y-axis
ax.set_xlabel('Temperature (K)')
ax.set_ylabel(r'n($cm^{-3}$)')
ax.set_title(f'Charge Carrier Density in {material_title} vs Temperature')

# Add a grid
ax.grid(False)

# Add a legend for the left y-axis
ax.legend(loc='center left', framealpha=0.4)

# Create a second y-axis on the right
ax2 = ax.twinx()

# Plot data on the right y-axis
ax2.scatter(hall_coefficient[:, 0], np.abs(hall_coefficient[:, 2]), color='purple', marker = 'x', label='$R^2_{A}$')
ax2.scatter(hall_coefficient[:, 0], np.abs(hall_coefficient[:, 4]), color='orange', marker = '+', label='$R^2_{B}$')

# Set labels for the right y-axis
ax2.set_ylabel('$R^2$ Value')

# Add a legend for the right y-axis
ax2.legend(loc='center right', framealpha=0.4)
ax2.set_ylim(0, 1)

# Show the plot
plt.show()

if plot_data == True:
    fig11.savefig(Path(path_out).joinpath(f'{plot_string}_n_vs_temperature.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)


## Mobility vs T

In [ ]:


# Create a figure and axis
fig12 = plt.figure(figsize=(10, 6))
ax = fig12.add_subplot(111)  # Create a single subplot

# Generate color map
c_val = plt.get_cmap('winter')(np.linspace(0, 1, 3))

# Create separate marker arrays for positive and negative fields stitched together
markers = ['x','o','*']


ax.scatter(hall_coefficient[:, 0], 1e4*np.divide(-hall_coefficient[:, 1],res_data[int(ctf[5]/2-1)::ctf[5],4]), marker=markers[0], color=c_val[0], label=f'Config_A')
ax.scatter(hall_coefficient[:, 0], 1e4*np.divide(-hall_coefficient[:, 3],res_data[int(ctf[5]/2-1)::ctf[5],4]), marker=markers[1], color=c_val[1], label=f'Config_B')
ax.scatter(hall_coefficient[:, 0], 1e4*np.divide(-hall_coefficient[:, 5],res_data[int(ctf[5]/2-1)::ctf[5],4]), marker=markers[2], color=c_val[2], label=f'Average')

# Set labels and title for the left y-axis
ax.set_xlabel('Temperature (K)')
ax.set_ylabel(r'$\mu$ (cm$^2$V$^{-1}$s$^{-1}$)')
ax.set_title(r'$\mu$' f' in {material_title} vs Temperature using Zero Field Resitivity')
#ax.set_ylim(-100, 0)

# Add a grid
ax.grid(False)

# Add a legend for the left y-axis
ax.legend(loc='center left', framealpha=0.4)

# Create a second y-axis on the right
ax2 = ax.twinx()

# Plot data on the right y-axis
ax2.scatter(hall_coefficient[:, 0], np.abs(hall_coefficient[:, 2]), color='purple', marker = 'x', label='$R^2_{A}$')
ax2.scatter(hall_coefficient[:, 0], np.abs(hall_coefficient[:, 4]), color='orange', marker = '+', label='$R^2_{B}$')

# Set labels for the right y-axis
ax2.set_ylabel('$R^2$ Value')

# Add a legend for the right y-axis
ax2.legend(loc='center right', framealpha=0.4)
ax2.set_ylim(0, 1)

# Show the plot
plt.show()

if plot_data == True:
    fig12.savefig(Path(path_out).joinpath(f'{plot_string}_mobility_vs_temperature.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)


## $\rho_{xy}$ vs $\rho_{xx}$
- Can have problems where with both holes and electrons the carriers can cancel out the measured hall voltage
- Need to check this?

In [ ]:
#ctf = [current_unique, temp_unique, field_unique, current_no, temp_no, field_no]

# Create a figure and axis
fig13 = plt.figure(figsize=(10, 6))
ax = fig13.add_subplot(111)  # Create a single subplot


# Generate color map
c_val = plt.get_cmap('winter')(np.linspace(0, 1, ctf[5]))

# Create separate marker arrays for positive and negative fields stiched together
markers = ['x'] * ctf[5]


for H_count, H in enumerate(ctf[2], start=0):
    index = t_count * ctf[5]
    # Plot rho_film vs Temp
    ax.plot(res_data[H_count::ctf[5],4]*1e8, hall_data[H_count::ctf[5],4]*1e8, marker=markers[H_count], color=c_val[H_count], label = f'H={np.round(H*10,decimals=0)} kOe')
            #ctf = [current_unique, temp_unique, field_unique, current_no, temp_no, field_no]


ax.set_ylabel(r'$\rho_{xy}\,(\mu \Omega \cdot cm)$')
ax.set_xlabel(r'$\rho_{xx}\,(\mu \Omega \cdot cm)$')
ax.set_title(r'$\rho_{xy}$ vs $\rho_{xx}$' f' for {material_title}')
# ax.set_xlim(0,100)
# ax.set_ylim(1425,1525)

# Add a grid
ax.grid(True)

# Add a legend
ax.legend(loc = 'best', framealpha=0.4)

# Show the plot
plt.show()

if plot_data == True:
    fig13.savefig(Path(path_out).joinpath(f'{plot_string}_rho_xx_vs_rho_xy.{fig_format}'), dpi=300, bbox_inches='tight', transparent=plot_transparency)

In [ ]:
if powerpoint_data == True:
    # Create a PowerPoint presentation object
    prs = Presentation()

    # Define a function to add a slide with a title and image
    def add_slide(prs, title, fig):
        slide_layout = prs.slide_layouts[5]  # Use a blank slide layout
        slide = prs.slides.add_slide(slide_layout)
        
        # Set background color with 40% transparency
        background = slide.background
        fill = background.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(200, 200, 200)

        # Add title with custom font
        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(24)
        title_shape.text_frame.paragraphs[0].font.bold = True
        title_shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
        title_shape.text_frame.paragraphs[0].font.name = 'Arial'
        title_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        # Save the figure as an image
        img_path = f"{title}.png"
        fig.savefig(img_path, dpi=300, bbox_inches='tight', transparent=True)

        
         # Calculate the left position to center the image
        slide_width = prs.slide_width
        image_width = Inches(8)
        left = (slide_width - image_width) / 2
        top = Inches(2)  # Position the figure lower down by 2 inches
        
        # Add the image to the slide
        pic = slide.shapes.add_picture(img_path, left, top, width=image_width)

    # List of figures and their corresponding titles
    figures = [
        (fig0, "Plot IV data to Check Linearity"),
        (fig1, "R-squared Analysis to Check Linearity of I-V data"),
        (fig2, "Resistivity Configuration Compararison"),
        (fig3, "Resistivity vs Temperature"),
        (fig4, "Magnetoresistance vs H"),
        (fig5, "Magnetoresistance Vs T"),
        (fig6, "Arhenius Plot"),
        (fig7, "R-squared Analysis to Check Linearity of Resistivity I-V data"),
        (fig8, "Hall resisitivity vs Temperature"),
        (fig9, "Hall resisitivity vs Applied Field"),
        (fig10, "Hall resisitivity vs Applied Field - Offset Removed"),
        (fig11, "Hall Coeff Vs T"),
        (fig12, "Mobility vs T"),
        (fig13, "ρ_xy vs ρ_xx")
    ]

    # Add each figure to the PowerPoint presentation
    for fig, title in figures:
        add_slide(prs, title, fig)

    # Save the PowerPoint presentation
    prs.save(Path(path_out).joinpath(f'{plot_string}_figures.pptx'))